# **Viña del Mar or Valparaíso, Who has better rating food?**
---
### Introduction
-------
Viña del Mar and Valparaíso, are widely know chilean cities in the coast of the central region of the country. Both cities are neighbors and share a variety of gastromic option in the opinion of people. Lately, have been seeing lots of signature cuisine restaurants, burgeries, bar \& pubs are opened, ofering a widely variety of alternative to taste different flavors.
Maybe that perception is wrong and such variety don't exist, and perhaps we are full of same kind restaurants.


#### Business Problem
-------
In this report I'll analyze if really exist variety in gastronomic offer and if the perception that Valparaiso has better restaurants is true, information that will be use as an orientarion for visitors of this two chilean cities.
Also this report will be useful for gastronomic investor, as a guidence allow them to know how really is the gastronomic offer in both cities.

##### Restrictions
-------
This report is not absolute information in any case and has to be taken only as a guide, since analysis and mesurements will based on data from www.foursquare.com and perhaps many restaurants haven't been rating yet.
The postal code in Chile is manage by www.correos.cl (Postal Chilean Company), but they don't provide open access to this information wich is necesary  to have a better granularity of the data. Therefore, i'll use the oficial coordinates of both cities location (center of the cities) assuming that in a radio of 3 km is possible to find the major concentration of turistic POI, which is true until certain point.

### Data Source
-------
The data will be gather from foursquare api using the coordinate of the cities obtained from geopy geocoder

----

#### Setting Up Work Enviroment

In [6]:
#! conda install -c conda-forge folium=0.5.0 --yes
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from geopy.geocoders import Nominatim
import folium
print('libraries needed ... loaded !!!')

libraries needed ... loaded !!!


In [11]:
# geting cities coordinate with geopy geolocator
city1 = 'Valparaíso, Chile'
city2 = 'Viña del mar, Chile'
cities = [city1, city2]
citiesCoordinates = []
geolocator = Nominatim(user_agent = 'My_notebook')
for city in cities:
    location = None
    while(location is None):
        location = geolocator.geocode(city)
    citiesCoordinates.append(location)

for i, city in enumerate(cities):
    print('{} \t in latitude: {}, longitude: {}'.format(city, citiesCoordinates[i].latitude, citiesCoordinates[i].longitude))

      


Valparaíso, Chile 	 in latitude: -33.0458456, longitude: -71.6196749
Viña del mar, Chile 	 in latitude: -33.0244535, longitude: -71.5517636


In [12]:
# foursquare API credentials
CLIENT_ID = 'PJ1XE1YHTFANS2Z3OT3PYMMEG2BBKOXNXXISOLGZ1VYH0SWQ'
SECRET_KEY = 'EQQC5KNXLTD4HJ5BRXOUB5XW22LHYJVIBVC5U3UGEAEPR5E4'
VERSION = '20200406'

In [25]:
radius = 3000
LIMIT = 100
#Exploring the data
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID,
    SECRET_KEY, 
    VERSION, 
    citiesCoordinates[0].latitude, 
    citiesCoordinates[0].longitude, 
    radius, LIMIT)
result = requests.get(url).json()
result

{'meta': {'code': 200, 'requestId': '5e8b6160bae9a2001b9fdea2'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Valparaíso',
  'headerFullLocation': 'Valparaíso',
  'headerLocationGranularity': 'city',
  'totalResults': 159,
  'suggestedBounds': {'ne': {'lat': -33.01884557299997,
    'lng': -71.58752445192725},
   'sw': {'lat': -33.07284562700003, 'lng': -71.65182534807276}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '54136e12498e5447674366c9',
       'name': 'Hotzenplotz',
       'location': {'address': 'Hector Calvo 331',
        'lat': -33.048339,
        'lng': -71.62251,
        'labeledLatLngs': [{'label': 'display',
          'lat': -33.048339,
          'lng': -71.622

In [68]:
# getting venues form cities
radius = 3000
LIMIT = 100

def getVenues(cities, coordinates, radius = 1000, limit = LIMIT):
    venueList = []
    for city, coord in zip(cities, coordinates):
        print(city)
        # create the API request url
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            SECRET_KEY, 
            VERSION, 
            coord.latitude, 
            coord.longitude, 
            radius, 
            LIMIT)
        
        #make GET request
        result = requests.get(url).json()['response']['groups'][0]['items']
        
        #return only relevant information from each nearby venue
        venueList.append([(
            city[0:city.find(',')],            
            coord.latitude, 
            coord.longitude,
            v['venue']['id'],
            v['venue']['name'],
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in result])
        nearbyVenues = pd.DataFrame([item for venueList in venueList for item in venueList])
        nearbyVenues.columns = ['City',
                                'City Latitude',
                                'City Longitude',
                                'id',
                                'Venue',
                                'Venue Latitude',
                                'Venue Longitude',
                                'Venue Category']
    return(nearbyVenues)

        
def getVenueScore(venue_id):
    url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(
        venue_id, 
        CLIENT_ID, 
        SECRET_KEY, 
        VERSION)
    result = requests.get(url).json()['response']['venue']['rating']
    return result
    
    

In [69]:
# create dataframe with venues per city

venuesVinaValpo = getVenues(cities, citiesCoordinates, radius = 3000)

Valparaíso, Chile
Viña del mar, Chile


In [70]:
venuesVinaValpo.head(20)

,City,City Latitude,City Longitude,id,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Valparaíso,-33.045846,-71.619675,54136e12498e5447674366c9,Hotzenplotz,-33.048339,-71.622510,German Restaurant
1,Valparaíso,-33.045846,-71.619675,4b61eaf6f964a5202a2b2ae3,Bogarín,-33.046735,-71.619637,Sandwich Place
2,Valparaíso,-33.045846,-71.619675,570bcfb2498eb339ee9d9d95,Sazón Nazca,-33.046749,-71.616066,Peruvian Restaurant
3,Valparaíso,-33.045846,-71.619675,56043f43498ec58350f23b9e,CasaPlan,-33.045029,-71.620884,Art Gallery
4,Valparaíso,-33.045846,-71.619675,4b647fbff964a52052b82ae3,Paseo Atkinson,-33.042663,-71.625263,Scenic Lookout
5,Valparaíso,-33.045846,-71.619675,54cd707f498e0e083b18ba42,Habrakadabra Sabores,-33.048186,-71.615182,Pizza Place
6,Valparaíso,-33.045846,-71.619675,4b5219d8f964a520dd6727e3,Museo de Historia Natural de Valparaíso,-33.046391,-71.621133,History Museum
7,Valparaíso,-33.045846,-71.619675,5155d374e4b0ec92c7aa4946,Espíritu Santo,-33.048899,-71.622355,Restaurant
8,Valparaíso,-33.045846,-71.619675,557c7d97498e95a6f8f191f8,Sazón Nazca,-33.047146,-71.619604,Peruvian Restaurant
9,Valparaíso,-33.045846,-71.619675,4e8209677ee61ffc04f3e5bd,"Bésame Mucho, Cafe Resto-Bar",-33.044912,-71.622231,Food


we need to explore the date in order to identify categories asociates with place that serve food

In [75]:
print(venuesVinaValpo.shape)
print('The are {} uniques categories.'. format(len(venuesVinaValpo['Venue Category'].unique())))
venuesVinaValpo.groupby('Venue Category').count()

(200, 8)
The are 87 uniques categories.


,City,City Latitude,City Longitude,id,Venue,Venue Latitude,Venue Longitude
Venue Category,,,,,,,
Art Gallery,1,1,1,1,1,1,1
Art Museum,2,2,2,2,2,2,2
Asian Restaurant,1,1,1,1,1,1,1
Assisted Living,1,1,1,1,1,1,1
Bagel Shop,1,1,1,1,1,1,1
Bakery,4,4,4,4,4,4,4
Bar,3,3,3,3,3,3,3
Beach,2,2,2,2,2,2,2
Bed & Breakfast,6,6,6,6,6,6,6


**From the 87 diferent categories we are interested only in the following:**

In [79]:
# define search values
searchValues = ['Restaurant', 'Bagel', 'Beer', 'Bar', 'Breakfast', 'Brewery', 'Burger', 'Coffee', 'Cupcake', 'Dessert', 'Diner', 'Launch', 'Pizza', 'Pub', 'Sandwich', 'Hot Dog', 'Ice Cream', 'Steakhose']

foodVenues = venuesVinaValpo[venuesVinaValpo['Venue Category'].str.contains('|'.join(searchValues))]
print(foodVenues.shape)
foodVenues.head(20)

(106, 8)


,City,City Latitude,City Longitude,id,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Valparaíso,-33.045846,-71.619675,54136e12498e5447674366c9,Hotzenplotz,-33.048339,-71.622510,German Restaurant
1,Valparaíso,-33.045846,-71.619675,4b61eaf6f964a5202a2b2ae3,Bogarín,-33.046735,-71.619637,Sandwich Place
2,Valparaíso,-33.045846,-71.619675,570bcfb2498eb339ee9d9d95,Sazón Nazca,-33.046749,-71.616066,Peruvian Restaurant
5,Valparaíso,-33.045846,-71.619675,54cd707f498e0e083b18ba42,Habrakadabra Sabores,-33.048186,-71.615182,Pizza Place
7,Valparaíso,-33.045846,-71.619675,5155d374e4b0ec92c7aa4946,Espíritu Santo,-33.048899,-71.622355,Restaurant
8,Valparaíso,-33.045846,-71.619675,557c7d97498e95a6f8f191f8,Sazón Nazca,-33.047146,-71.619604,Peruvian Restaurant
11,Valparaíso,-33.045846,-71.619675,518122d77ab46f49cf6d2f85,Casa Cervecera Altamira,-33.044041,-71.626262,Brewery
12,Valparaíso,-33.045846,-71.619675,4f17434be4b07e1b48077406,Pizzeria Ecuador,-33.047225,-71.625047,Pizza Place
13,Valparaíso,-33.045846,-71.619675,4f987a1ee4b06b674a55a6fa,Fauna Bar Restaurant,-33.043673,-71.627015,Bar
14,Valparaíso,-33.045846,-71.619675,4b8525a1f964a5201e4e31e3,Malandrino,-33.043878,-71.628771,Pizza Place


In [83]:
# We need to obtain the rating of each place

# creation of column rating on dataframe with defualt value -1
foodVenues['Rating'] = -1
foodVenues.head()


,City,City Latitude,City Longitude,id,Venue,Venue Latitude,Venue Longitude,Venue Category,Rating
0,Valparaíso,-33.045846,-71.619675,54136e12498e5447674366c9,Hotzenplotz,-33.048339,-71.622510,German Restaurant,-1
1,Valparaíso,-33.045846,-71.619675,4b61eaf6f964a5202a2b2ae3,Bogarín,-33.046735,-71.619637,Sandwich Place,-1
2,Valparaíso,-33.045846,-71.619675,570bcfb2498eb339ee9d9d95,Sazón Nazca,-33.046749,-71.616066,Peruvian Restaurant,-1
5,Valparaíso,-33.045846,-71.619675,54cd707f498e0e083b18ba42,Habrakadabra Sabores,-33.048186,-71.615182,Pizza Place,-1
7,Valparaíso,-33.045846,-71.619675,5155d374e4b0ec92c7aa4946,Espíritu Santo,-33.048899,-71.622355,Restaurant,-1


In [96]:
for i, venue in enumerate(foodVenues['Venue'].to_list()):
    venue_id = foodVenues.iloc[i]['id']
    foodVenues.at[i,'Rating'] = getVenueScore(venue_id)

foodVenues.head(25)

/opt/conda/envs/Python36/lib/python3.6/site-packages/pandas/core/frame.py:2823: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.loc[index, col] = value


,City,City Latitude,City Longitude,id,Venue,Venue Latitude,Venue Longitude,Venue Category,Rating
0,Valparaíso,-33.045846,-71.619675,54136e12498e5447674366c9,Hotzenplotz,-33.048339,-71.622510,German Restaurant,9.0
1,Valparaíso,-33.045846,-71.619675,4b61eaf6f964a5202a2b2ae3,Bogarín,-33.046735,-71.619637,Sandwich Place,8.0
2,Valparaíso,-33.045846,-71.619675,570bcfb2498eb339ee9d9d95,Sazón Nazca,-33.046749,-71.616066,Peruvian Restaurant,8.0
5,Valparaíso,-33.045846,-71.619675,54cd707f498e0e083b18ba42,Habrakadabra Sabores,-33.048186,-71.615182,Pizza Place,8.2
7,Valparaíso,-33.045846,-71.619675,5155d374e4b0ec92c7aa4946,Espíritu Santo,-33.048899,-71.622355,Restaurant,8.5
8,Valparaíso,-33.045846,-71.619675,557c7d97498e95a6f8f191f8,Sazón Nazca,-33.047146,-71.619604,Peruvian Restaurant,8.7
11,Valparaíso,-33.045846,-71.619675,518122d77ab46f49cf6d2f85,Casa Cervecera Altamira,-33.044041,-71.626262,Brewery,8.2
12,Valparaíso,-33.045846,-71.619675,4f17434be4b07e1b48077406,Pizzeria Ecuador,-33.047225,-71.625047,Pizza Place,8.3
13,Valparaíso,-33.045846,-71.619675,4f987a1ee4b06b674a55a6fa,Fauna Bar Restaurant,-33.043673,-71.627015,Bar,8.6
14,Valparaíso,-33.045846,-71.619675,4b8525a1f964a5201e4e31e3,Malandrino,-33.043878,-71.628771,Pizza Place,8.6


In [90]:
# save the data to csv
foodVenues.to_csv()

['Hotzenplotz',
 'Bogarín',
 'Sazón Nazca',
 'Habrakadabra Sabores',
 'Espíritu Santo',
 'Sazón Nazca',
 'Casa Cervecera Altamira',
 'Pizzeria Ecuador',
 'Fauna Bar Restaurant',
 'Malandrino',
 'Restaurant Nazca',
 'Hostería Espíritu Santo',
 'Trigomar',
 'Restaurant La Concepción',
 'La Riviera',
 'Cocina Puerto',
 'Kuukai',
 'Samsara',
 "Brecon's",
 'La Dulcería',
 'La Belle Epoque',
 'Amor Porteño',
 'Le Petit Quer',
 'El Bar del Tío',
 'La  Mangiata',
 'Emporio La Rosa',
 'Ápice',
 'Leche Café',
 'Govindas',
 'La Cevichería',
 'Kiosko Roca',
 'Montealegre Restaurante',
 'La Joya Sánguches',
 'Restaurant El Peral',
 'Restaurant Chino Chui Yin',
 'Pasteleria Plaza',
 'sultan kebap',
 'Pizza Pronto',
 'Restaurante San Carlos',
 'Mestizzo',
 'Empanadas Dorite',
 'Arte y Salero',
 'Cafe República',
 'Casa Volante Hostal',
 'Chillout',
 'La Piazza Pizzería "El Almendral"',
 'Emporio Santo Domingo',
 'Panzoni',
 'Fuente de Soda Cevasco',
 'Bogarín',
 'La Mexicana',
 'Déjà Vu',
 'Empanadas

In [95]:
foodVenues.at[2,'Rating'] = -1

In [94]:
foodVenues.at[]

,City,City Latitude,City Longitude,id,Venue,Venue Latitude,Venue Longitude,Venue Category,Rating
0,Valparaíso,-33.045846,-71.619675,54136e12498e5447674366c9,Hotzenplotz,-33.048339,-71.622510,German Restaurant,-1
1,Valparaíso,-33.045846,-71.619675,4b61eaf6f964a5202a2b2ae3,Bogarín,-33.046735,-71.619637,Sandwich Place,-1
2,Valparaíso,-33.045846,-71.619675,570bcfb2498eb339ee9d9d95,Sazón Nazca,-33.046749,-71.616066,Peruvian Restaurant,100
5,Valparaíso,-33.045846,-71.619675,54cd707f498e0e083b18ba42,Habrakadabra Sabores,-33.048186,-71.615182,Pizza Place,-1
7,Valparaíso,-33.045846,-71.619675,5155d374e4b0ec92c7aa4946,Espíritu Santo,-33.048899,-71.622355,Restaurant,-1
